# 画像分類のビルトインアルゴリズムを使って猫の写真を分類する



# はじめに
***

このノートブックは、Amazon SageMaker Ground Truth でラベリングした結果を使って画像分類モデルを作成するためのノートブックです。このノートブックを使用する前に、Amazon SageMaker Ground Truth を使って画像のラベリングが完了している必要があります。

Amazon SageMaker Ground Truth を使った画像のラベリング方法については [こちらの記事](https://aws.amazon.com/jp/builders-flash/202003/sagemaker-groundtruth-cat/) をご参照ください。

# 初期設定
***

## 環境設定

まずはじめに、**みなさまの環境に合わせて以下の変数の値を変更します。**各変数に設定すべき値は以下の手順で確認可能です。

1. **labeling_job_name**<br>
Amazon SageMaker コンソールの左側のメニューの [ラベリングジョブ] をクリックし、ラベリングジョブ一覧を表示させます。ここの [名前] 欄にあるのがラベリングジョブ名です。該当のラベリングジョブ名をコピーしてこちらに設定します。
1. **your_augmented_manifest_file**<br>
上記手順で表示したラベリングジョブ一覧の中から使用したいラベリングジョブをクリックします。[ラベリングジョブの概要] 部分に [出力データセットの場所] というリンクがあるのでそちらをクリックします。そこから manifests -> output とフォルダをたどると output.manifest があります。そちらをクリックし、[コピーパス] というボタンをクリックすると、output.manifest ファイルの S3 パスがコピーされるので、それをペーストします。
1. **object_categories**<br>
上記手順で　[出力データセットの場所] というリンクをクリック後、annotation-tool フォルダをクリックすると、data.json というファイルがあります。こちらをクリックし [ダウンロード] ボタンをクリックしてダウンロードします。テキストエディタなどでこのファイルを開くと、ラベリング時に使用したラベル名と順番がわかりますので、こちらを参考に設定します。順番さえ同じであれば、アルファベットのラベル名をカタカナなどにしてこちらに設定しても問題ありません。
1. **width, height**<br>
上記手順1 で表示したラベリングジョブ一覧の中から使用したいラベリングジョブをクリックします。[ラベル付きデータセットオブジェクト] 部分に画像が表示されるので、そちらをクリックするとその画像が保存されている S3 パスがわかります。こちらから画像をダウンロードして、画像のサイズを確認できます。

変数の設定が完了したら、セルを選択した状態で Shift + Enter を実行するか、上にある Run ボタンをクリックするとセルが実行されます。セルというのは、ソースコードが書かれたテキスト領域のことです。セルを実行すると、セルの左側にある In [ ]: のカッコの中に数字が表示されます。セルの処理が実行中の場合は数字ではなく * が表示されます。

In [5]:
# Amazon SageMaker Ground Truth のラベリングジョブ名
labeling_job_name = 'yujimurata-car-image-labeling'

# Amazon SageMaker Ground Truth のラベリング結果の output.manifest の S3パス
your_augmented_manifest_file = 's3://yujimurata-car-image-dataset/labeled_data/yujimurata-car-image-labeling/manifests/output/output.manifest'

# Amazon SageMaker Ground Truth で使用したラベルの名前。ラベリング時と順番を揃える必要があります。
object_categories = ['Crossover', 'Truck', 'SUV', 'Car', 'Others']


# 学習に使用する画像（Amazon SageMaker Ground Truth でラベリングした画像）のサイズ
width = 800
height = 600
channel = 3 # 基本的にここは変更する必要はありません。カラー画像であればたいてい RGB の 3チャネルです。

## SageMaker を使うための設定

ここでは、AWS のサービスを使う上で必要な設定を行います。こちらは変更の必要はありません。どんどんセルを実行していきましょう。

* このノートブックインスタンスで使用しているロールの情報の取得
* デフォルト S3 バケット名の取得。こちらに学習したモデルなどを保存します
* The Amazon SageMaker の画像分類アルゴリズムの docker イメージパスの取得

In [6]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'ic-transfer-learning'

arn:aws:iam::430477916104:role/service-role/AmazonSageMaker-ExecutionRole-20211013T152104
CPU times: user 286 ms, sys: 5.8 ms, total: 292 ms
Wall time: 585 ms


In [18]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/image-classification:1


# 学習データの準備
***

ここでは、Amazon SageMaker Ground Truth でラベリングした結果を、学習用と検証用に分けて S3 にアップロードします。こちらは変更の必要はありません。

In [19]:
# S3 から output.manifest をダウンロード
tmp_manifest = '/tmp/output.manifest'
! aws s3 cp {your_augmented_manifest_file} {tmp_manifest}

download: s3://yujimurata-car-image-dataset/labeled_data/yujimurata-car-image-labeling/manifests/output/output.manifest to ../../../tmp/output.manifest


In [21]:
# output.manifest を学習用と検証用に分割

import random

with open(tmp_manifest) as f:
    data = [l.rstrip() for l in f.readlines()]
    
line_count = len(data)
num_training_samples = int(line_count * 0.7)

index = list(range(0, line_count))
random.shuffle(index)

train_data  = data[:num_training_samples]
valid_data = data[num_training_samples:]

train_manifest = '/tmp/train.manifest'
valid_manifest = '/tmp/valid.manifest'
with open(train_manifest, 'w') as f:
    for d in train_data:
        f.write("%s\n" % d)
with open(valid_manifest, 'w') as f:
    for d in valid_data:
        f.write("%s\n" % d)

In [22]:
# 学習用と検証用の manifest ファイルを S3 にアップロード

import os
s3path = os.path.dirname(your_augmented_manifest_file)
train_manifest_s3 = s3path + '/train.manifest'
valid_manifest_s3 = s3path + '/valid.manifest'
! aws s3 cp {train_manifest} {train_manifest_s3}
! aws s3 cp {valid_manifest} {valid_manifest_s3}

upload: ../../../tmp/train.manifest to s3://yujimurata-car-image-dataset/labeled_data/yujimurata-car-image-labeling/manifests/output/train.manifest
upload: ../../../tmp/valid.manifest to s3://yujimurata-car-image-dataset/labeled_data/yujimurata-car-image-labeling/manifests/output/valid.manifest


# 画像分類モデルの学習
***

ここまでで、必要な準備が完了しました。ここでは、画像分類モデルを作ります。始めに ``sagemaker.estimator.Estimator`` オブジェクトを作ります。この estimator から学習ジョブを立ち上げます。こちらも特に変更が必要な部分はありません。

## 学習パラメータ

モデルを学習するにあたって、2種類のパラメータを設定する必要があります。1つは学習ジョブの設定、もう 1つは画像分類モデルのハイパーパラメータです。

学習ジョブの設定には以下のようなものがあります。

* **学習インスタンスの数**: 学習ジョブで使用するインスタンスの数です。1 よりも大きな値を設定することで分散学習を実行することができます。ただし、画像分類のビルトインアルゴリズムは Pipe モード時の分散学習に対応していません。本ノートブックのように Amazon SageMaker Ground Truth が出力した拡張マニフェストファイルを使用する場合はファイル転送を Pipe モードにする必要があるため、分散学習を使用することはできませんのでご注意ください。
* **学習インスタンスタイプ**: 学習ジョブで使用するインスタンスタイプです。画像分類のビルトインアルゴリズムでは、p2 または p3 の GPU インスタンスのみをサポートします。
* **出力パス**: 学習ジョブの出力先となる S3 のパスです、

In [23]:
# 学習ジョブの設定

s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
ic = sagemaker.estimator.Estimator(training_image,
                                      role,
                                      instance_count=1,
                                      instance_type='ml.p2.xlarge',
                                      volume_size = 50,
                                      max_run = 360000,
                                      input_mode = 'Pipe',
                                      output_path=s3_output_location,
                                      sagemaker_session=sess)

画像分類モデルのハイパーパラメータには以下のようなものがあります。

* **num_layers**: モデルネットワークの層の数です。このサンプルでは 18 を使用しますが、[18、34、50、101、152、200] の中から選択できます。
* **use_pretrained_model**: 1 を設定することで学習済みモデルを使って転移学習をすることができます。
* **image_shape**: 学習に使用する画像のサイズです。チャネル数、高さ、幅、の順で設定します。実際の画像サイズより大きな値を設定しないようにしてください。
* **num_classes**: 画像分類の分類ラベルの数です。Imagenet は 1000 クラスの分類ですが、実際に使用したいラベルの数に合わせてクラスの数を変更します。2 匹の猫を分類するモデルを作る場合は、2 を設定します。
* **num_training_samples**: 学習に用いる画像の枚数を設定します。
* **mini_batch_size**: 各ミニバッチで使用する学習サンプル数です。分散学習では、バッチごとに使用される学習サンプル数は N * mini_batch_size となり、Nは訓練を実行するホストの数を意味します。
* **epochs**: 学習エポック数です。
* **learning_rate**: 学習率です。
* **precision_dtype**: 学習データの精度 (default: float32) です。float16 に設定した場合、学習は mixed_precision モードとなり、float32 よりも高速に学習します。


In [24]:
# 画像分類モデルのハイパーパラメータの設定

ic.set_hyperparameters(num_layers=18,
                             use_pretrained_model=1,
                             image_shape = str(channel)+','+str(height)+','+str(width),
                             num_classes=len(object_categories),
                             num_training_samples=num_training_samples,
                             mini_batch_size=4,
                             epochs=20,
                             learning_rate=0.01,
                             precision_dtype='float32')

## 入力データの設定

学習データの形式を定義します。今回は Amazon SageMaker Ground Truth で作成した拡張マニフェスト形式のファイルを使用するための設定をします。

In [25]:
# 拡張マニフェスト形式のファイルを学習データとして使用するよう設定

train_data = sagemaker.inputs.TrainingInput(train_manifest_s3,
                                        distribution='FullyReplicated',
                                        content_type='application/x-recordio',
                                        record_wrapping='RecordIO',
                                        s3_data_type='AugmentedManifestFile',
                                        attribute_names=['source-ref', labeling_job_name]) 
validation_data = sagemaker.inputs.TrainingInput(valid_manifest_s3,
                                        distribution='FullyReplicated',
                                        content_type='application/x-recordio',
                                        record_wrapping='RecordIO',
                                        s3_data_type='AugmentedManifestFile',
                                        attribute_names=['source-ref', labeling_job_name]) 
data_channels = {'train': train_data, 'validation': validation_data}

## 学習の開始

定義した estimator の fit() を実行して学習ジョブを開始します。ログの最後には、

```
Training seconds: 255
Billable seconds: 255
```
のように、学習ジョブに対して課金される対象となる時間が秒単位で表示されます。


> **注意： ResourceLimitExceeded というエラーが出たら**<br>
作ったばかりの新しい AWSアカウントを使用する場合など、ノートブック実行中に ResourceLimitExceeded というエラーが出ることがあります。その場合は [サポートセンター](https://console.aws.amazon.com/support/home#/) から 「ケースの作成」 をクリックしたのち 「サービス制限の緩和」を選択し、以下の設定でケースを作成します。
> - 制限タイプ：SageMaker
> - リージョン：米国東部（バージニア北部）　※実際に使用するリージョンを選択してください
> - リソースタイプ：学習実行時にエラーが出た場合は「SageMaker のトレーニング」、推論時にエラーが出た場合は「SageMaker のホスト」
> - 制限：使用するインスタンスタイプ
> - 申請理由の説明：ハンズオンで使用するため、など

In [26]:
ic.fit(inputs=data_channels, logs=True)

2021-10-13 07:29:14 Starting - Starting the training job...
2021-10-13 07:29:38 Starting - Launching requested ML instancesProfilerReport-1634110154: InProgress
......
2021-10-13 07:30:38 Starting - Preparing the instances for training...............
2021-10-13 07:33:07 Downloading - Downloading input data...
2021-10-13 07:33:43 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[10/13/2021 07:33:48 INFO 140285547784000] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[10/13/2021 07:33:48 INFO 140285547784000] Merging with provided configuration fro

UnexpectedStatusException: Error for Training job image-classification-2021-10-13-07-29-14-006: Failed. Reason: ClientError: Invalid image found. Please ensure data is in one of the expected image formats


# 推論 - 猫を見分けてみよう！

***

## 推論環境の起動
モデルを学習しただけでは何も起こりません。作ったモデルを使って、写真に写っているのがどちらの猫なのかを推論してみましょう。estimator の deploy() を実行して推論環境を立ち上げます。ログにはしばらく `-` が表示され、処理が完了したら `!` が表示されます。**こちらの処理が完了するまでは 5分から 10分程度かかります。**

ResourceLimitExceeded というエラーが出たら、[学習の開始](#学習の開始) に記載の手順でサービス制限の緩和を行ってください。

In [27]:
from sagemaker.serializers import IdentitySerializer

ic_classifier = ic.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.m4.xlarge',
                                          serializer = IdentitySerializer(content_type='application/x-image'))

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-ap-northeast-1-430477916104/ic-transfer-learning/output/image-classification-2021-10-13-07-29-14-006/output/model.tar.gz.

## 推論用画像の準備

S3 に保存してある画像をダウンロードして、どちらの猫の写真なのか推論してみます。画像は `/tmp` にダウンロードされます。**S3 のパスは、みなさまの環境に合わせて書き換えてください。**

In [ ]:
# 画像を S3 からローカル（ノートブックインスタンス）の /tmp/にダウンロード

!aws s3 cp "s3://cat-image-classification/cats/DSC_0022.jpg" /tmp/

ダウンロードした画像を確認します。**`file_name` は先ほどダウンロードした画像の名前に書き換えてください。**

In [ ]:
# 画像の表示

file_name = '/tmp/DSC_0022.jpg'
from IPython.display import Image
Image(file_name)  

## 写真に写っている猫を見分ける（推論の実行）

立ち上げた推論環境に写真を送って、写真にどちらの猫が写っているのかを見分けます。推論結果としては、各ラベル（猫の名前）と、写真に写っているのがその猫である確率（probability）が出力されます。そこから、最も probability が高いものを最終結果とします。使用する画像を変えて、何度か推論を試してみましょう！

**注意:** 結果がいまひとつの場合、学習に使用する画像を増やす、ハイパーパラメータの epoch の数を増やす、その他のハイパーパラメータチューニングを行うなどで精度の向上が見込めます。

In [ ]:
# ダウンロードした写真を読み込んで、推論環境に送り推論結果を取得し、最終結果を表示

import json
import numpy as np

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
    
result = json.loads(ic_classifier.predict(payload))
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)

print("写っているのは " + object_categories[index] + "です！ （probability - " + str(round(result[index]*100, 1)) + '%）')

# リソースの削除
***
不要な課金を防ぐため、使用しないリソースは削除しましょう。なお、削除したデータは戻せないのでご注意ください。本ノートブックを実行することで使用される AWS サービスは以下の通りです。削除方法は [こちらの記事](https://aws.amazon.com/jp/builders-flash/202005/sagemaker-groundtruth-cat/) の「リソースの削除」の章をご参照ください。
- Amazon S3 バケット
- Amazon SageMaker ノートブックインスタンス
- Amazon SageMaker エンドポイント

## 推論環境（エンドポイント）の削除

推論環境は立ち上げている間課金されるので、ありったけの猫の写真を見分け終わったら、次のコマンドを実行して推論環境を削除しましょう。delete_endpoint() でエンドポイントを削除することができます。

Amazon SageMaker のコンソールからエンドポイント一覧の画面にいき、そちらからエンドポイントを削除することも可能です。

In [ ]:
ic_classifier.delete_endpoint()